In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('musicbrainz-cmudb2020.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print(tables)
print(len(tables))

[('area',), ('artist',), ('artist_alias',), ('artist_credit_name',), ('artist_credit',), ('artist_type',), ('gender',), ('language',), ('medium',), ('medium_format',), ('release',), ('release_info',), ('release_status',), ('work',), ('work_type',)]
15


Observe that we have 15 tables in this database.

## Q1 (q1_sample)

The purpose of this query is to make sure that the formatting of your output matches exactly the formatting of our auto-grading script.

<b>Details:</b> List all types of work ordered by type ascendingly.


In [3]:
# Lets have a look at all columns in work_type table

cursor.execute("PRAGMA table_info('work_type')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

id
name
description


In [4]:
output = pd.read_sql_query("""SELECT name FROM work_type ORDER BY name""", conn)
output

,name
0,Aria
1,Audio drama
2,Ballet
3,Beijing opera
4,Cantata
5,Concerto
6,Incidental music
7,Madrigal
8,Mass
9,Motet


<h2>Q2 (q2_long_name):


List works with the longest name of each type.

<b>Details:</b> For each work type, find works that have the longest names. There might be cases where there is a tie for the longest names - in that case, return all of them. Display work names and corresponding type names, and order it according to work type (ascending) and use work name (ascending) as tie-breaker.

In [5]:
q2 = pd.read_sql_query(
    ''' select work.name,
    work_type.name
from work
    inner join (
        select max(length(work.name)) as max_length,
            work.type as type
        from work
        group by work.type
    ) as newtable on newtable.max_length = length(work.name)
    and work.type = newtable.type
    inner join work_type on work.type = work_type.id
order by work.type asc,
    work.name asc;''', conn)
q2

,name,name
0,"Don Carlo: Atto III, scena 2. N°14. Morte di R...",Aria
1,Macbeth: Atto III. Ballo: Allegro vivacissimo ...,Ballet
2,"Cantata profana for tenor, baritone, double ch...",Cantata
3,"Concerto for Piano and Strings, op. 12: II. Th...",Concerto
4,Sonata violino solo representativa (Representa...,Sonata
5,France Fall Suite: The Maginot Line / Paris De...,Suite
6,"Madrigali, Libro 8, ""Madrigali guerrieri, et a...",Madrigal
7,"Missa Salisburgensis, mass for 2 double chorus...",Mass
8,"Totus amore languens (motet for solo voice, tw...",Motet
9,"Genoveva op. 81: Akt 4 Nr. 18 Rezitativ, Terze...",Opera


<h2> Q3 (q3_old_music_nations):

List top 10 countries with the most classical music artists (born or started before 1850) along with the number of associated artists.



<b>Details:</b> Print country and number of associated artists before 1850. For example, Russia|191. Sort by number of artists in descending order.


In [6]:
q3 = pd.read_sql_query(
    '''select a2.name,
    count(*) as c
from artist a1
    inner join area a2 on a1.area = a2.id
where begin_date_year < 1850
group by a1.area
order by c desc
limit 10;''', conn);
q3

,name,c
0,Germany,1543
1,France,1169
2,Italy,1049
3,United Kingdom,766
4,Austria,355
5,United States,344
6,Spain,270
7,Russia,191
8,England,181
9,Czech Republic,156


<h2> Q4 (q4_dubbed_smash):


List the top 10 dubbed artist names with the number of dubs.


<b>Details:</b> Count the number of distinct names in artist_alias for each artist in the artist table, and list only the top ten who's from the United Kingdom and started after 1950 (not included). Print the artist name in the artist table and the number of corresponding distinct dubbed artist names in the artist_alias table.

In [7]:
q4 = pd.read_sql_query(
    '''Select artist.name,
    count(distinct artist_alias.name) as num
From artist
    inner join artist_alias on artist.id = artist_alias.artist
Where artist.begin_date_year > 1950
    and area = 221
Group by artist.id
Order by num desc
Limit 10;''', conn);

q4

,name,num
0,The Beatles,35
1,Judas Priest,16
2,The KLF,14
3,Orchestral Manoeuvres in the Dark,14
4,Deep Purple,14
5,National Philharmonic Orchestra,13
6,T. Rex,13
7,Led Zeppelin,13
8,Heller & Farley,12
9,Chris Barber’s Jazz Band,12


<h2> Q5 (q5_vinyl_lover):

List the distinct names of releases issued in vinyl format by the British band Coldplay.



<b>Details:</b> Vinyl format includes ALL vinyl dimensions excluding VinylDisc. Sort the release names by release date ascendingly.


In [8]:
q5 = pd.read_sql_query(
    '''select distinct r1.name as rname
    from artist_credit_name a1
    inner join artist_credit a2 on a1.artist_credit = a2.id
    inner join release r1 on a2.id = r1.artist_credit
    inner join release_info r2 on r1.id = r2.release
    inner join medium m1 on r1.id = m1.release
    inner join medium_format m2 on m1.format = m2.id
where a1.name = 'Coldplay'
    and m2.name like '%Vinyl'
order by date_year,
    date_month,
    date_day;''', conn)

q5

,rname
0,Brothers & Sisters
1,Trouble
2,Parachutes
3,Shiver
4,A Rush of Blood to the Head
5,Remixes
6,Speed of Sound
7,Fix You
8,Talk - The Remixes
9,The Singles 1999–2006


<h2>Q6 (q6_old_is_not_gold):

Which decades saw the most number of official releases? List the number of official releases in every decade since 1900. Like 1970s|57210.



<b>Details:</b> Print all decades and the number of official releases. Releases with different issue dates or countries are considered different releases. Print the relevant decade in a fancier format by constructing a string that looks like this: 1970s. Sort the decades in decreasing order with respect to the number of official releases and use decade (descending) as tie-breaker. Remember to exclude releases whose dates are NULL.


In [9]:
q6 = pd.read_sql_query(
'''Select decade,
    count(*) as cnt
from (
        select (CAST((date_year / 10) as int) * 10) || 's' as decade
        from release
            inner join release_info on release.id = release_info.release
        where release.status = 1
            and date_year >= 1900
    )
Group by decade
Order by cnt desc,
    decade desc;''', conn)

q6

,decade,cnt
0,2010s,1628648
1,2000s,686375
2,2020s,556090
3,1990s,360612
4,1980s,131551
5,1970s,57210
6,1960s,30084
7,1950s,8739
8,1900s,1797
9,1940s,1659


<h2>Q7 (q7_release_percentage):


List the month and the percentage of all releases issued in the corresponding month all over the world in the past year. Display like 2020.01|5.95.


<b>Details:</b> The percentage of releases for a month is the number of releases issued in that month divided by the total releases in the past year from 07/2019 to 07/2020, both included. Releases with different issue dates or countries are considered different releases. Round the percentage to two decimal places using ROUND(). Sort by dates in ascending order.


In [11]:
q7 = pd.read_sql_query(
'''with past_year_release (year, month) as (
    select date_year,
        date_month
    from release_info r1
        inner join release r2 on r1.release = r2.id
    where (
            (
                date_year = 2019
                and date_month >= 7
            )
            or (
                date_year = 2020
                and date_month <= 7
            )
        )
)
select cast(year as varchar) || '.' || (
        case
            when month < 10 then '0'
            else ''
        end
    ) || cast(month as varchar) as date,
    round(
        count(*) * 100.0 / (
            select count(*)
            from past_year_release
        ),
        2
    )
from past_year_release
group by date
order by date;''', conn)

q7

,date,"round(\n count(*) * 100.0 / (\n select count(*)\n from past_year_release\n ),\n 2\n )"
0,2019.07,3.73
1,2019.08,7.40
2,2019.09,6.14
3,2019.10,5.97
4,2019.11,5.78
5,2019.12,3.56
6,2020.01,5.95
7,2020.02,6.58
8,2020.03,12.03
9,2020.04,16.48


<h2> Q8 (q8_collaborate_artist):


List the number of artists who have collaborated with Ariana Grande.


<b>Details:</b> Print only the total number of artists. An artist is considered a collaborator if they appear in the same artist_credit with Ariana Grande. The answer should include Ariana Grande herself.


In [12]:
q8 = pd.read_sql_query(
'''Select count(distinct artist)
From artist_credit_name
Where artist_credit in (
        select artist_credit
        from artist_credit_name
        where name = 'Ariana Grande'
    );''', conn)

q8

,count(distinct artist)
0,111


<h2> Q9 (q9_dre_and_eminem):


List the rank, artist names, along with the number of collaborative releases of Dr. Dre and Eminem among other most productive duos (as long as they appear in the same release) both started after 1960 (not included). Display like [rank]|Dr. Dre|Eminem|[# of releases].


<b>Details:</b> For example, if you see a release by A, B, and C, it will contribute to three pairs of duos: A|B|1, A|C|1, and B|C|1. You will first need to calculate a rank of these duos by number of collaborated releases (release with artist_credit shared by both artists) sorted descendingly, and then find the rank of Dr. Dre and Eminem. Only releases in English are considered. Both artists should be solo artists. All pairs of names should have the alphabetically smaller one first. Use artist names (asc) as tie breaker.


<b>Hint:</b> Artist aliases may be used everywhere. When doing aggregation, using artist ids will ensure you get the correct results. One example entry in the rank list is 9|Benj Pasek|Justin Paul|27


In [13]:
q9 = pd.read_sql_query(
'''with duos_list (id1, id2, count) as (
    select a1.artist as id1,
        a2.artist as id2,
        count(*) as c
    from artist_credit_name a1
        inner join artist_credit_name a2 on a1.artist_credit = a2.artist_credit
        inner join release r on a2.artist_credit = r.artist_credit
        inner join artist a3 on a1.artist = a3.id
        inner join artist a4 on a2.artist = a4.id
        inner join artist_type a5 on a3.type = a5.id
        inner join artist_type a6 on a4.type = a6.id
        inner join language l on r.language = l.id
    where a3.name < a4.name
        and a5.name = "Person"
        and a6.name = "Person"
        and l.name = 'English'
        and a3.begin_date_year > 1960
        and a4.begin_date_year > 1960
    group by a1.artist,
        a2.artist
)
select *
from (
        select row_number () over (
                order by count desc,
                    a1.name,
                    a2.name
            ) as rank,
            a1.name as name1,
            a2.name as name2,
            count
        from duos_list d
            inner join artist a1 on d.id1 = a1.id
            inner join artist a2 on d.id2 = a2.id
    )
where name1 = 'Dr. Dre'
    and name2 = 'Eminem';''', conn)

q9

,rank,name1,name2,count
0,41,Dr. Dre,Eminem,15


<h2> Q10 (q10_around_the_world):


Concat all dubbed names of The Beatles using comma-separated values(like "Beetles, fab four").


<b>Details:</b> Find all dubbed names of artist "The Beatles" in artist_alias and order them by id (ascending). Print a single string containing all the dubbed names separated by commas.
Hint: You might find CTEs useful. 

In [14]:
q10 = pd.read_sql_query('''with c as (
      select row_number() over (
                  order by c.id asc
            ) as seqnum,
            c.name as name
      from artist_alias c
            join artist on c.artist = artist.id
      where artist.name = 'The Beatles'
),
flattened as (
      select seqnum,
            name as name
      from c
      where seqnum = 1
      union all
      select c.seqnum,
            f.name || ', ' || c.name
      from c
            join flattened f on c.seqnum = f.seqnum + 1
)
select name
from flattened
order by seqnum desc
limit 1;''', conn)

q10

,name
0,"Beatles, Beetles, fab four, 더 비틀즈, ザ・ビートルズ, Lo..."
